To use pytesseract :  
1. It works only on images so convert all pdf to images using `pdf2image`.
2. `Pytesseract` is just a wrapper it need tesseract to be installed on your system by ```brew install tesseract``` for mac.
3. 

In [6]:
import pytesseract
from pdf2image import convert_from_path
from PIL import Image
import os

def extract_text_from_scanned_pdf(pdf_path, tesseract_cmd=None):
    """Extract text from a scanned PDF using Tesseract OCR."""
    if tesseract_cmd:
        pytesseract.pytesseract.tesseract_cmd = tesseract_cmd
    # Convert PDF to images
    images = convert_from_path(pdf_path)
    text = ""
    for i, image in enumerate(images):
        page_text = pytesseract.image_to_string(image)
        text += f"\n--- Page {i+1} ---\n{page_text}"
    return text

pdf_file = 'data/first_year/chemistry/Mid_Sem/13-14 chem3.pdf'
# If tesseract is not in PATH, specify its location, e.g. '/usr/local/bin/tesseract'
tesseract_path = "/opt/homebrew/bin/tesseract"

extracted_text = extract_text_from_scanned_pdf(pdf_file, tesseract_cmd=tesseract_path)
print(extracted_text)


--- Page 1 ---
i.
2.

a!
4.
ae

Course: - B.Tech. 1° Yr, 11 Sem, 2" mid Term Examination, 2013-14

Subject: Engg. Chemistry (AHC-101) Uni. Roll No:-
Time:-90 Minutes Total Marks:-20
Note:-

Answer any all questions from Sec A, Any three from Sec B and Any three from Sec C.

All questions of the particular group should be answered collectively at one place. All parts of a question (a, b, etc.) should
be answered at one place.

Answer should be brief and to-the-point and be supplemented with neat sketches.

Any missing or wrong data may be assumed suitably giving proper justification.

Figures on the right-hand side margin indicate full marks.

Section-A
All ons. 5X1=5 marks
(I) Write down the relation between the rate constant and temp. of a reaction

(I!) Calculate the bond order of He," ion.

(II!) Define a semiconductor.

(IV) What is point or centre of symmetry in a compound?
(V) Write the average composition of gobar gas.

Section-B
Note: Attem Three Questions. 3X2=6 marks
(1) Der